<a href="https://colab.research.google.com/github/firststef/APLN-WSD/blob/master/WSD_Romanian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install spark-nlp==4.2.3 pyspark==3.2.1
#!pip3 install pybuilder
#!git clone https://github.com/paudan/opennlp_python.git
#!cd opennlp_python && pyb 
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
!pip install -U setuptools wheel
!pip install -U spacy
!python -m spacy download ro_core_news_md

In [6]:
import sparknlp
import nltk
import spacy
spark = sparknlp.start()
nltk.download('punkt')
spnlp = spacy.load('ro_core_news_md')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

def get_word_embeddings(sentence):
  documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

  tokenizer = Tokenizer() \
  .setInputCols("document") \
  .setOutputCol("token")

  embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d","ro") \
  .setInputCols(["document", "token"]) \
  .setOutputCol("embeddings")

  pipeline = Pipeline(stages=[documentAssembler, tokenizer, embeddings])

  data = spark.createDataFrame([[sentence]]).toDF("text")

  result = pipeline.fit(data).transform(data)

  return result

In [1]:
result = get_word_embeddings("Propozitie de exemplu")
result.select('embeddings').show(truncate=False)

NameError: ignored

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

def truncate_sentence_semantic(sentence, word):
  toks = word_tokenize(sentence)
  index = toks.index(word)
  left = index - 3
  left = 0 if left < 0 else left
  right = index + 4
  right = len(toks) if right >= len(toks) else right
  return " ".join(toks[left:right])

truncate_sentence_semantic("Ca hobby, obisnuiesc sa merg la sala unde trag cat pot de mine!", "sala")

'sa merg la sala unde trag cat'

In [ ]:
def get_sintactic_semantic(sentence):
  doc = spnlp(sentence)
  return ([x.pos_ for x in doc], [(x.text,x.label_) for x in doc.ents])

print(get_sintactic_semantic("El este Ștefan, vine din Romania."))

(['PRON', 'AUX', 'PROPN', 'PUNCT', 'AUX', 'ADP', 'PROPN', 'PUNCT'], [('El', 'PERSON'), ('Ștefan', 'GPE'), ('Romania', 'GPE')])


In [ ]:
import nltk
#nltk.download('mte_teip5')
print(nltk.corpus.multext_east.tagged_words("oana-ro.xml"))

[('Într-', '#Spsay'), ('o', '#Tifsr'), ...]


In [ ]:
import nltk, nltk.classify.util, nltk.metrics
from nltk.classify import MaxentClassifier
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.classify import MaxentClassifier

from nltk.corpus import senseval

# from nltk.corpus import movie_reviews

# def word_feats(words):
#  return dict([(word, True) for word in words])

# negids = movie_reviews.fileids('neg')
# posids = movie_reviews.fileids('pos')

# negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
# posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# negcutoff = len(negfeats)*3//4
# poscutoff = len(posfeats)*3//4

# trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(trainfeats, algorithm,max_iter=3)

classifier.show_most_informative_features(10)

# all_words = nltk.FreqDist(word for word in movie_reviews.words())
# top_words = set(all_words.keys()[:300])

# def word_feats(words):
#     return {word:True for word in words if word in top_words}

  ==> Training (3 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.500
             2          -0.69252        0.953
         Final          -0.69190        0.953
  -0.000 magnificent==True and label is 'neg'
  -0.000 insulting==True and label is 'pos'
  -0.000 vulnerable==True and label is 'neg'
  -0.000 uninvolving==True and label is 'pos'
  -0.000 avoids==True and label is 'neg'
  -0.000 outstanding==True and label is 'neg'
  -0.000 astounding==True and label is 'neg'
  -0.000 fascination==True and label is 'neg'
  -0.000 ludicrous==True and label is 'pos'
  -0.000 affecting==True and label is 'neg'


In [ ]:
# am de parsat sens-eval 3 si pus in algoritmul de antrenare cumva